In [1]:
import sys
import numpy as np
import open3d as o3d
sys.path.insert(1, "../data")
from utils import readData, readPointCloud

In [2]:
ground_truth = readData("../data/01.txt")
ground_truth = ground_truth[:77][:]

In [3]:
def computeTransformation(point_ind):
    ground_truth = readData("../data/01.txt")
    ground_truth = ground_truth[:77][:]
    T = ground_truth[point_ind][:]
    T = np.reshape(T, (3,4))
    b = np.array([0,0,0,1])
    T = np.vstack ((T,b)) 
    return T

In [4]:
def computePoseCameraFrame(point_ind):
    file = "../data/01/" + str(point_ind).zfill(6) + ".bin"
    pcd = readPointCloud(file)
#     valid = []
#     for j in range(len(pcd)):
#         dist = np.linalg.norm(pcd[j][:3])
#         if dist <= 50 and dist >= 10:
#             valid.append(pcd[j])
#     pcd = np.asarray(valid)
    npts = pcd.shape[0]
    lidar_to_cam = np.array([[0,-1,0,0],[0,0,-1,0],[1,0,0,0],[0,0,0,1]])
    pcd = np.hstack((pcd[:,:3],np.ones((npts,1))))
    poses = np.dot(np.asarray(pcd),lidar_to_cam.T)
    poses = np.dot(poses,computeTransformation(point_ind).T)
    return poses

In [5]:
pcd_net = o3d.geometry.PointCloud()
for i in range(77):
    poses = computePoseCameraFrame(i)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(poses[:,:3])
    pcd_net += pcd
downpcd = pcd_net.voxel_down_sample(voxel_size=1)
o3d.visualization.draw_geometries([downpcd])

Firstly, and most importantly, for point cloud registrations, we used the transformation matrix details given in the file 01.txt, reshaped them to form matrices of the following type:
\begin{align}
T = [R|t]_{3 \times 4}
\end{align}

Since our points are in 3D space ($Q$), our transformed point cloud ($P$) will look like this:
\begin{align}
P = RQ + t
\end{align}

Instead of having to perform matrix multiplication and vector addition at every iteration in the process, we made the following changes to our data representation for easier computation - each point in 3D was converted into its homogeneous coordinate representation by appending a 1 at the end of the vector, i.e.,
\begin{align}
Q_{3 \times 1} = \begin{bmatrix} x \\ y \\ z \end{bmatrix} \implies Q_{hc} = \begin{bmatrix} x \\ y \\ z \\ 1\end{bmatrix} 
\end{align}
and the transformation matrix was changed to look like this {SE(3) representation}:



\begin{align}
T_{hc} = \begin{bmatrix} R & t\\ \mathbf{0}^T & 1 \end{bmatrix}
\end{align}

Because of this, our new point cloud representation looks like this:
\begin{align}
P_{hc} = T_{hc}Q_{hc} = \begin{bmatrix} x' \\ y' \\ z' \\ 1 \end{bmatrix} \implies P = \begin{bmatrix} x' \\ y' \\ z' \end{bmatrix}
\end{align}

Now, since our frame of concern is the camera frame, and the points are observed in the LiDAR frame, it is necessary to apply the transformation on the points so that we go from the LiDAR frame into the camera frame. This transformation is applied before the above transformations to bring us to the camera frame from the LiDAR frame and aid us in the next part of the assignment as well. The transformation matrix for the same is:\
![Frame Orientations](../README/v1.png)
\begin{align}
Q^{lidar} = T_{camera}^{lidar} Q^{camera} \\ \\
T_{camera}^{lidar} = \begin{bmatrix} R_{camera}^{lidar} & t_{camera}^{lidar} \\ \mathbf{0}^T & 1 \end{bmatrix} \\ \\
R_{camera}^{lidar} = \begin{bmatrix} \hat{x}_{cam} \hat{x}_{lidar} & \hat{y}_{cam} \hat{x}_{lidar} & \hat{z}_{cam} \hat{x}_{lidar} \\ \hat{x}_{cam} \hat{y}_{lidar} & \hat{y}_{cam} \hat{y}_{lidar} & \hat{z}_{cam} \hat{y}_{lidar} \\ \hat{x}_{cam} \hat{z}_{lidar} & \hat{y}_{cam} \hat{z}_{lidar} & \hat{z}_{cam} \hat{z}_{lidar} \end{bmatrix} \\ \\
R_{camera}^{lidar} = \begin{bmatrix} 0 & -1 & 0 \\ 0 & 0 & -1 \\ 1 & 0 & 0 \end{bmatrix} \\ \\
t_{camera}^{lidar} = \begin{bmatrix} 0 \\ 0 \\ 0 \end{bmatrix} \\ \\
\therefore T_{camera}^{lidar} = \begin{bmatrix} 0 & -1 & 0 & 0 \\ 0 & 0 & -1 & 0 \\ 1 & 0 & 0 & 0 \\ 0 & 0 & 0 & 1 \end{bmatrix} \\ 
\end{align}

Just so that we don't fall into any confusion in the process, since we changed the coordinate representation from being a 3x1 vector to a 4x1 vector, we changes the above transformation matrix into a 4x4 matrix, similar to how the above $T_{hc}$ was written, just that the $t$ vector here is the $\mathbf{0}_{3 \times 1}$ vector.


Contributions by team members:

Sudhansh: Setting up transformations in homogeneous system, data restructuring, report\
Dipanwita: Pose computations, plotting